# _____________________
#          Achtung bei "Run All"
# _____________________

# Erstellen des Batch-Sets

In [1]:
# Für diverse Datenoperationen
import pandas as pd
from accelerate.commands.config.update import description

# Zum Laden von Datasets von Huggingface
from datasets import load_dataset, Dataset as DS

import requests as r

import json

from tqdm import tqdm


Laden des Datasets

In [2]:
dataset = load_dataset("terminAl-thesis-2025/combined_dataset")
dataset_df = pd.DataFrame(dataset["train"])

dataset_df

,nl_prompt,command,check,__index_level_0__
0,What is the total number of addresses for each...,"SELECT country, state_province_county, COUNT(*...",True,0
1,Find all addresses in the United States that d...,SELECT * FROM addresses WHERE country = 'Unite...,True,1
2,What is the count of addresses in Europe by ci...,"SELECT city, COUNT(*) AS address_count FROM ad...",True,2
3,"Get a list of all countries, along with the nu...","SELECT country, COUNT(*) AS address_count FROM...",True,3
4,What is the total number of addresses for each...,"SELECT city, COUNT(*) AS total_addresses FROM ...",True,4
...,...,...,...,...
174974,List all running services,systemctl list-units --type=service,True,25418
174975,Find and replace 'foo' with 'bar' in all .txt ...,sed -i 's/foo/bar/g' *.txt,True,25419
174976,Check if port 8080 is in use,netstat -tulnp | grep 8080,True,25420
174977,Display real-time disk I/O stats,iostat -dx 1,True,25421


Hinzufügen einer eindeutigen ID (sequenzielle ID)

In [3]:
dataset_df["id"] = (dataset_df.index +1).astype(str)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/175 [00:00<?, ?ba/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/datasets/terminAl-thesis-2025/trainings_set/commit/1498b7701aab0b5c06051ee383ecab6c16957a97', commit_message='Upload dataset', commit_description='', oid='1498b7701aab0b5c06051ee383ecab6c16957a97', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/terminAl-thesis-2025/trainings_set', endpoint='https://huggingface.co', repo_type='dataset', repo_id='terminAl-thesis-2025/trainings_set'), pr_revision=None, pr_num=None)

In [4]:
dataset_df.head(2)

,nl_prompt,command,check,__index_level_0__,id
0,What is the total number of addresses for each...,"SELECT country, state_province_county, COUNT(*...",True,0,1
1,Find all addresses in the United States that d...,SELECT * FROM addresses WHERE country = 'Unite...,True,1,2


Definition des Systemprompts

In [5]:
base_prompt = '''Du bist ein spezialisierter Assistent, der natürlichsprachliche Anfragen zu CLI-Befehlen (Command Line Interface) oder SQL-Abfragen in ein strukturiertes JSON-Format übersetzt. Deine Antwort muss immer exakt im folgenden JSON-Format sein:

{{
 "command": "der vollständige und korrekte Befehl/die SQL-Abfrage",
 "tool": "bash/sql",
 "risk_level": "low/medium/high",
 "description": "Kurze Beschreibung des Befehls oder der Abfrage",
 "detailed_description": "Längere Erklärung, was der Befehl tut und wie die Parameter funktionieren",
 "potential_consequences": ["mögliche Auswirkungen oder Risiken des Befehls", "weitere Auswirkung wenn zutreffend"]
}}

Konvertiere die folgende natürlichsprachliche Anfrage in das JSON-Format. Verwende dabei ausschließlich die Informationen aus der Anfrage und dem zugehörigen Befehl:

ANFRAGE: "{0}"
BEFEHL: "{1}"

Richtlinien:
1. Bestimme den Wert für "tool" korrekt basierend auf dem Befehl: Verwende "sql" für SQL-Abfragen und "bash" für Linux/Bash-Befehle.
2. Achte unbedingt auf die korrekte Syntax des Commands. Korrigiere den Command falls nötig (z.B. fehlendes Semikolon bei SQL-Abfragen hinzufügen).
3. Gib für SQL-Abfragen "low" als risk_level an, es sei denn, es handelt sich um DELETE, DROP, ALTER oder UPDATE-Operationen.
4. Für CLI-Befehle bewerte das Risiko basierend auf der Möglichkeit von Datenverlust oder Systemänderungen.
5. Die "description" sollte prägnant sein (max. 1 Satz).
6. Die "detailed_description" sollte nicht mehr als drei Sätze umfassen.
7. Liste unter "potential_consequences" alle relevanten möglichen Auswirkungen auf, oder ein leeres Array [], wenn keine nennenswerten Risiken bestehen.
8. Antworte IMMER mit einem gültigen JSON-Objekt, ohne Einleitung oder Abschluss.
9. Verwende keine Formatierungszeichen wie "\\n", "\\t" oder andere Escape-Sequenzen im JSON.
10. Stelle sicher, dass das JSON vollständig gültig ist und direkt als Python-Dictionary geladen werden kann.
11. Halte dich strikt an doppelte Anführungszeichen für Keys und String-Werte, wie es im JSON-Standard vorgeschrieben ist.
12. Gib "description", "detailed_description" und "potential_consequences" IMMER in deutscher Sprache zurück, auch wenn die ursprüngliche Anfrage auf Englisch ist.
13. Verwende bei der Übersetzung eine klare und präzise deutsche Fachsprache.

Ergänzende Richtlinien zur Bestimmung des "risk_level":

- Verwende "low", wenn der Befehl:
  * rein lesend ist (z.B. `ls`, `cat`, `SELECT` ohne `JOIN` auf große Tabellen),
  * keine Änderungen an Daten, Systemkonfigurationen oder Dateistrukturen vornimmt,
  * typischerweise keine Auswirkungen auf andere Prozesse oder Nutzer hat.

- Verwende "medium", wenn der Befehl:
  * potenziell Änderungen an Dateien, Datenbankeinträgen oder Konfigurationen durchführt, jedoch reversibel oder mit geringer Auswirkung ist (z.B. `mv`, `cp`, `touch`, `UPDATE` mit WHERE),
  * Netzwerkverbindungen aufbaut oder Systemprozesse anstößt (z.B. `curl`, `ping`, `systemctl restart`),
  * selektive Datenbearbeitung in SQL betrifft, jedoch nicht strukturverändernd ist.

- Verwende "high", wenn der Befehl:
  * irreversible Änderungen am System oder an Daten verursacht (z.B. `rm -rf /`, `DROP TABLE`, `ALTER DATABASE`),
  * kritische Dienste beeinträchtigt oder beendet (z.B. `kill -9 1`, `shutdown`),
  * ohne weitere Rückfrage systemweite Auswirkungen haben kann,
  * Sicherheitsrisiken birgt, z.B. durch Datenfreigabe, Prozessbeeinflussung oder das Löschen von Benutzerkonten.

Beziehe immer auch Kontext und typische Folgen mit ein. Wenn Unsicherheit besteht, ist ein konservativerer (höherer) Risikowert zu bevorzugen.'''


Erweitere Dataframe

In [6]:
dataset_df["json"] = ""

In [7]:
dataset_df.columns

Index(['nl_prompt', 'command', 'check', '__index_level_0__', 'id', 'json'], dtype='object')

Erstellen der Requests, mit Testsplit (um die API zu testen und Kosten abzuschätzen)

In [8]:
test_requests = {}
test_requests_mini = {}
requests = {"batch_1":{}}
model = "llama3.1:8b-instruct-q5_K_M"
#model = "llama3.3:70b-instruct-q4_K_M"
batch_size = 1000
batch = 1

for index, row in dataset_df.iterrows():
    nl_prompt = row['nl_prompt']
    command = row['command']
    prompt = base_prompt.format(nl_prompt, command)
    request = {
        "model": model,
        "prompt": prompt,
        "format": "json",
        "stream": False
    }

    if index < batch_size:
        batch_id = "batch_{}".format(batch)
        requests[batch_id][index] = request
    elif index == batch_size:
        batch += 1
        batch_size += 1000
        batch_id = "batch_{}".format(batch)
        requests[batch_id] = {}
        requests[batch_id][index] = request

print(len(test_requests))
print(len(requests))

0
175


In [9]:
requests["batch_1"]

{0: {'model': 'llama3.1:8b-instruct-q5_K_M',
  'prompt': 'Du bist ein spezialisierter Assistent, der natürlichsprachliche Anfragen zu CLI-Befehlen (Command Line Interface) oder SQL-Abfragen in ein strukturiertes JSON-Format übersetzt. Deine Antwort muss immer exakt im folgenden JSON-Format sein:\n\n{\n "command": "der vollständige und korrekte Befehl/die SQL-Abfrage",\n "tool": "bash/sql",\n "risk_level": "low/medium/high",\n "description": "Kurze Beschreibung des Befehls oder der Abfrage",\n "detailed_description": "Längere Erklärung, was der Befehl tut und wie die Parameter funktionieren",\n "potential_consequences": ["mögliche Auswirkungen oder Risiken des Befehls", "weitere Auswirkung wenn zutreffend"]\n}\n\nKonvertiere die folgende natürlichsprachliche Anfrage in das JSON-Format. Verwende dabei ausschließlich die Informationen aus der Anfrage und dem zugehörigen Befehl:\n\nANFRAGE: "What is the total number of addresses for each country, grouped by state/province/county?"\nBEFEHL:

# Synthetische Trainingsdaten mit Llama

In [10]:
dataset_df

,nl_prompt,command,check,__index_level_0__,id,json
0,What is the total number of addresses for each...,"SELECT country, state_province_county, COUNT(*...",True,0,1,
1,Find all addresses in the United States that d...,SELECT * FROM addresses WHERE country = 'Unite...,True,1,2,
2,What is the count of addresses in Europe by ci...,"SELECT city, COUNT(*) AS address_count FROM ad...",True,2,3,
3,"Get a list of all countries, along with the nu...","SELECT country, COUNT(*) AS address_count FROM...",True,3,4,
4,What is the total number of addresses for each...,"SELECT city, COUNT(*) AS total_addresses FROM ...",True,4,5,
...,...,...,...,...,...,...
174974,List all running services,systemctl list-units --type=service,True,25418,174975,
174975,Find and replace 'foo' with 'bar' in all .txt ...,sed -i 's/foo/bar/g' *.txt,True,25419,174976,
174976,Check if port 8080 is in use,netstat -tulnp | grep 8080,True,25420,174977,
174977,Display real-time disk I/O stats,iostat -dx 1,True,25421,174978,


In [14]:
errors = {}
for id, batch in tqdm(requests.items(), desc="Befrage Ollama", unit="Anfrage"):
    results = {}
    print(id)
    for index, req in batch.items():
        try:
            response = r.post("http://localhost:11434/api/generate", json=req)
            response.raise_for_status()
            result = response.json()
            results[index] = result.get("response")


        except Exception as e:
            errors[index] = str(e)
    with open(f"../data/json_results/results_{id}.json", "w") as f:
        json.dump(results, f)

print(len(errors))

Befrage Ollama:   0%|          | 0/3 [00:00<?, ?Anfrage/s]

batch_173


Befrage Ollama:   0%|          | 0/3 [1:10:19<?, ?Anfrage/s]


KeyboardInterrupt: 

In [51]:
results

{0: '{\n "command": "SELECT country, state_province_county, COUNT(*) AS total_addresses FROM addresses GROUP BY country, state_province_county;",\n "tool": "sql",\n "risk_level": "low",\n "description": "Ermittelt die Gesamtzahl der Adressen für jedes Land, gruppiert nach Bundesland/Kreis.",\n "detailed_description": "Diese SQL-Abfrage zählt die Anzahl der Adressen in einer Tabelle und gruppiert das Ergebnis nach Ländern und deren Unterteilungen. Sie dient zur Informationsbeschaffung und verändert keine Daten. Die Abfrage ist somit rein lesend und birgt daher kein Risiko für Datenverlust oder -manipulation.",\n "potential_consequences": []\n}',
 1: '{\n "command": "SELECT * FROM addresses WHERE country = \'United States\' AND state_province_county IS NULL;",\n "tool": "sql",\n "risk_level": "low",\n "description": "Findet alle Adressen in den Vereinigten Staaten, bei denen kein Bundesstaat oder Landkreis angegeben ist.",\n "detailed_description": "Diese SQL-Abfrage durchsucht eine Tabe

Kombiniere die Resultate und ergänze sie mit dem Dataframe